<a href="https://colab.research.google.com/github/Kai1219/colab-python-data-practice/blob/main/News.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 口罩API

In [ ]:
# @title
import requests
import json

# 利用 requests 對 API 來源發送一個請求
url = "https://raw.githubusercontent.com/kiang/pharmacies/master/json/points.json"
response = requests.get(url)

# 將請求回應的內容存成一個字串格式
d = response.text

# 將長得像 json 格式的字串解析成字典或列表
data = json.loads(d)

In [ ]:
# @title
# 將取回來的資料利用 Python 語法計算各地區的藥局數量

med_count = {}

for d in data["features"]:
  conunty = d["properties"]["county"]
  if conunty not in med_count:
    med_count[conunty] = 0
  med_count[conunty] += 1

print(med_count)
# {'臺北市': 339, '': 49, '高雄市': 422, '臺中市': 420, '臺南市 ...}

{'臺北市': 339, '': 49, '高雄市': 422, '臺中市': 420, '臺南市': 271, '基隆市': 57, '新竹市': 39, '嘉義市': 67, '新北市': 501, '桃園市': 264, '新竹縣': 46, '宜蘭縣': 76, '苗栗縣': 56, '彰化縣': 179, '南投縣': 67, '雲林縣': 129, '嘉義縣': 84, '屏東縣': 140, '澎湖縣': 11, '花蓮縣': 46, '臺東縣': 23, '金門縣': 6, '連江縣': 1}


In [ ]:
# @title
# 計算出每個地區的成人剩餘口罩數量，並且將結果從大到小排列
mask_count = {}
total = 0
for d in data["features"]:
  conunty = d["properties"]["county"]
  if conunty not in mask_count:
    mask_count[conunty] = 0
  mask_count[conunty] += d["properties"]["mask_adult"]
  total += d["properties"]["mask_adult"]

mask_count = dict(sorted(mask_count.items(),key=lambda item:item[1],reverse=True))

print(mask_count)
# {'新北市': 670270, '臺中市': 536710, '高雄市': 424170, '臺南市': 393900, '桃園市': 331940, '臺北市': 322490, '彰化縣': 219400, '雲林縣': 196990 ...}

{'新北市': 670270, '臺中市': 536710, '高雄市': 424170, '臺南市': 393900, '桃園市': 331940, '臺北市': 322490, '彰化縣': 219400, '雲林縣': 196990, '屏東縣': 184250, '苗栗縣': 131080, '嘉義縣': 113180, '嘉義市': 107840, '宜蘭縣': 106900, '南投縣': 91240, '基隆市': 80100, '': 69210, '花蓮縣': 57330, '新竹縣': 51350, '新竹市': 45630, '臺東縣': 28510, '金門縣': 15510, '澎湖縣': 12600, '連江縣': 7930}


In [ ]:
# @title
# 將「各地區的藥局數量」存入資料庫的 `pharmacies` 表格，欄位包含「地區」、「數量」、「新增時間」
import requests
import json
import sqlite3
import datetime

# 利用 requests 取得口罩API資料
url = "https://raw.githubusercontent.com/kiang/pharmacies/master/json/points.json"
response = requests.get(url)
d = response.text
data = json.loads(d)

med_count = {}

for d in data["features"]:
  conunty = d["properties"]["county"]
  if conunty not in med_count:
    med_count[conunty] = 0
  med_count[conunty] += 1

# 建立資料庫
con = sqlite3.connect("mask.db")
c = con.cursor()

# 建立pharmacies資料表
c.execute('''CREATE TABLE IF NOT EXISTS pharmacies
           (conunty TEXT,quantity INTEGER,create_time DATETIME)''')
c.execute("DELETE FROM pharmacies")
con.commit()

for med in med_count.items():
  t = datetime.datetime.now()
  c.execute(f"INSERT INTO pharmacies VALUES ('{med[0]}',{med[1]},'{t}')")

con.commit()

c.execute("SELECT * FROM pharmacies")
print(c.fetchall())
con.close()

[('臺北市', 339, '2025-08-03 12:25:12.438209'), ('', 49, '2025-08-03 12:25:12.438359'), ('高雄市', 422, '2025-08-03 12:25:12.438394'), ('臺中市', 420, '2025-08-03 12:25:12.438407'), ('臺南市', 271, '2025-08-03 12:25:12.438432'), ('基隆市', 57, '2025-08-03 12:25:12.438443'), ('新竹市', 39, '2025-08-03 12:25:12.438457'), ('嘉義市', 67, '2025-08-03 12:25:12.438471'), ('新北市', 501, '2025-08-03 12:25:12.438484'), ('桃園市', 264, '2025-08-03 12:25:12.438496'), ('新竹縣', 46, '2025-08-03 12:25:12.438508'), ('宜蘭縣', 76, '2025-08-03 12:25:12.438517'), ('苗栗縣', 56, '2025-08-03 12:25:12.438527'), ('彰化縣', 179, '2025-08-03 12:25:12.438537'), ('南投縣', 67, '2025-08-03 12:25:12.438550'), ('雲林縣', 129, '2025-08-03 12:25:12.438562'), ('嘉義縣', 84, '2025-08-03 12:25:12.438598'), ('屏東縣', 140, '2025-08-03 12:25:12.438618'), ('澎湖縣', 11, '2025-08-03 12:25:12.438631'), ('花蓮縣', 46, '2025-08-03 12:25:12.438642'), ('臺東縣', 23, '2025-08-03 12:25:12.438653'), ('金門縣', 6, '2025-08-03 12:25:12.438662'), ('連江縣', 1, '2025-08-03 12:25:12.438679')]


In [ ]:
# 將「各地區的剩餘成人口罩」存入資料庫的 `masks` 表格，欄位包含「地區」、「剩餘成人口罩數量」、「新增時間」
import sqlite3
import json
import datetime
import requests

# 利用 requests 取得口罩API資料
url = "https://raw.githubusercontent.com/kiang/pharmacies/master/json/points.json"
response = requests.get(url)
d = response.text
data = json.loads(d)

mask_count = {}

for d in data["features"]:
  conunty = d["properties"]["county"]
  if conunty not in mask_count:
    mask_count[conunty] = 0
  mask_count[conunty] += d["properties"]["mask_adult"]

# 建立資料庫
con = sqlite3.connect("mask.db")
c = con.cursor()

# 刪掉舊表格
c.execute("DROP TABLE IF EXISTS masks")
# 建立masks資料表
c.execute('''CREATE TABLE IF NOT EXISTS masks (conunty TEXT,adult_masks INTEGER,create_time DATETIME)''')
# 刪除表格中的所有資料列
c.execute("DELETE FROM masks")
con.commit()


for county,adult_masks in mask_count.items():
  t = datetime.datetime.now()
  # print(f"INSERT INTO masks VALUES ('{county}',{adult_masks},{t})")
  c.execute("INSERT INTO masks VALUES (?, ?, ?)",(county,adult_masks,t))
  con.commit()

c.execute("SELECT * FROM masks")
print(c.fetchall())

con.close()

[('臺北市', 322490, '2025-08-03 12:31:21.105281'), ('', 69210, '2025-08-03 12:31:21.110640'), ('高雄市', 424170, '2025-08-03 12:31:21.115099'), ('臺中市', 536710, '2025-08-03 12:31:21.119711'), ('臺南市', 393900, '2025-08-03 12:31:21.126441'), ('基隆市', 80100, '2025-08-03 12:31:21.131327'), ('新竹市', 45630, '2025-08-03 12:31:21.136096'), ('嘉義市', 107840, '2025-08-03 12:31:21.140775'), ('新北市', 670270, '2025-08-03 12:31:21.145774'), ('桃園市', 331940, '2025-08-03 12:31:21.150749'), ('新竹縣', 51350, '2025-08-03 12:31:21.156080'), ('宜蘭縣', 106900, '2025-08-03 12:31:21.161180'), ('苗栗縣', 131080, '2025-08-03 12:31:21.166179'), ('彰化縣', 219400, '2025-08-03 12:31:21.171022'), ('南投縣', 91240, '2025-08-03 12:31:21.178620'), ('雲林縣', 196990, '2025-08-03 12:31:21.185695'), ('嘉義縣', 113180, '2025-08-03 12:31:21.192868'), ('屏東縣', 184250, '2025-08-03 12:31:21.199758'), ('澎湖縣', 12600, '2025-08-03 12:31:21.206111'), ('花蓮縣', 57330, '2025-08-03 12:31:21.213152'), ('臺東縣', 28510, '2025-08-03 12:31:21.219684'), ('金門縣', 15510, '2025-08

# 新聞語料關鍵字分析 jieba

In [ ]:
import requests
from bs4 import BeautifulSoup

# 利用 requests 對 API 來源發送一個請求
response = requests.get('https://udn.com/news/breaknews/1')
soup = BeautifulSoup(response.text)

news = []

for link in soup.find_all('h3', class_='rounded-thumb__title')[:4]:
  news_url = link.a['href']
  news_response = requests.get('https://udn.com' + news_url)
  news_soup = BeautifulSoup(news_response.text)
  news_content = news_soup.find('div', class_='article-content__paragraph').text.strip().replace('\n', ' ')
  news.append(news_content)

print(news)